## Scikit Learn
Source: [Getting Started with Scikit Learn](https://scikit-learn.org/stable/getting_started.html) <br>

Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection, model evaluation, and many other utilities.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)

X = [[ 1,  2,  3],  # 2 samples, 3 features
     [11, 12, 13]]

display(pd.DataFrame(X,
                     index=["Sample0", "Sample1"],
                     columns=["Feature0", "Feature1", "Feature2"]))

y = [0, 1]  # Classes of each sample

clf.fit(X, y)

The fit method generally accepts 2 inputs:
1. The samples matrix (or design matrix) X. The size of X is typically (n_samples, n_features), which means that samples are represented as rows and features are represented as columns.
2.  The target values y which are real numbers for regression tasks, or integers for classification (or any other discrete set of values). For unsupervised learning tasks, y does not need to be specified. y is usually a 1d array where the i th entry corresponds to the target of the i th sample (row) of X.

Once the estimator is fit, it can be used to predict the target values of new (unseen!) data. No re-training is needed for this.


In [ ]:
print("Prediction of Training Data:\n", clf.predict(X))  # predict classes of the training data


print("Unseen Data Prediction:\n", clf.predict([[4, 5, 6], [14, 15, 16]])) # predict classes of new data

### Transformers and Pre-Processors
Machine learning workflows are often composed of different parts. A typical pipeline consists of a pre-processing step that transforms or imputes the data, and a final predictor that predicts target values.

In scikit-learn, pre-processors and transformers follow the same API as the estimator objects (they actually all inherit from the same BaseEstimator class). The transformer objects don’t have a predict method but rather a transform method that outputs a newly transformed sample matrix X:

In [ ]:
from sklearn.preprocessing import StandardScaler

X = [[0, 15],
     [1, -10]]
print("Unscaled X\n", X)
# scale data according to computed scaling values
scaled_X = StandardScaler().fit(X).transform(X)
print("Scaled X (mean=0, std=1)\n", scaled_X)

### Pipelines: Chaining pre-processors and estimators
Transformers and estimators (predictors) can be combined together into a single unifying object: a Pipeline. The pipeline offers the same API as a regular estimator: it can be fitted and used for prediction with fit and predict. As we will see later, using a pipeline will also prevent you from data leakage, i.e. disclosing some testing data in your training data.

In the following example, we load the Iris dataset, split it into train and test sets, and compute the accuracy score of a pipeline on the test data:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# create a pipeline object
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)
display(pipe)

# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# fit the whole pipeline
pipe.fit(X_train, y_train)

# we can now use it like any other estimator
print("Accuracy Score:\n", 
      accuracy_score(pipe.predict(X_test), y_test))

### Model Evaluation
Fitting a model to some data does not entail that it will predict well on unseen data. This needs to be directly evaluated. We have just seen the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) helper that splits a dataset into train and test sets, but scikit-learn provides many other tools for model evaluation, in particular for cross-validation.

<img src="CrossValidation.png" alt="Cross Validation" width="400"/>

We here briefly show how to perform a 5-fold [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) procedure, using the cross_validate helper. Note that it is also possible to manually iterate over the folds, use different data splitting strategies, and use custom scoring functions. Please refer to the Scikit Learn [User Guide](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) for more details:

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

X, y = make_regression(n_samples=1000, random_state=0)
lr = LinearRegression()

result = cross_validate(lr, X, y)  # defaults to 5-fold CV
print("Test Score:\n", result['test_score'])

### Hyperparameters
All estimators have parameters (often called hyper-parameters in the literature) that can be tuned. The generalization power of an estimator often critically depends on a few parameters. For example a [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) has a **n_estimators parameter** that determines the number of trees in the forest, and a **max_depth** parameter that determines the maximum depth of each tree. Quite often, it is not clear what the exact values of these parameters should be since they depend on the data at hand.

Scikit-learn provides tools to automatically find the best parameter combinations (via cross-validation). In the following example, we randomly search over the parameter space of a random forest with a [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) object. When the search is over, the [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) behaves as a RandomForestRegressor that has been fitted with the best set of parameters. Read more in the [User Guide](https://scikit-learn.org/stable/modules/grid_search.html#grid-search)

Be very careful about seperating your training and testing sets. For instance if you apply pre-processing, only scale based on the training-set, not the whole dataset. See [Search over pipeline](https://scikit-learn.org/stable/modules/grid_search.html#composite-grid-search)

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# define the parameter space that will be searched over
param_distributions = {'n_estimators': randint(1, 5),
                       'max_depth': randint(5, 10)}


# now create a searchCV object and fit it to the data
search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0),
                            n_iter=5,
                            param_distributions=param_distributions,
                            random_state=0)

search.fit(X_train, y_train)

display(search)

print("Best Hyperparameters:\n", search.best_params_)

# the search object now acts like a normal random forest estimator
# with max_depth=9 and n_estimators=4
print("Score:\n",search.score(X_test, y_test))

### Examples
There are many examples of Scikit Learn in action: <br>
[Gallery of SciKit Learn Examples](https://scikit-learn.org/stable/auto_examples/index.html#general-examples)